# Detect Hallucinations
This notebook shows two different methods of detecting hallucinations:
  1) Asking an LLM to rate the output, and list any potential hallucinations.  This is more of a qualitative judgment.
  2) Use an LLM specifically trained to detect hallucinations.  Here we use the [Vectara model](https://huggingface.co/vectara/hallucination_evaluation_model?) from HuggingFace.
  
This notebook is divided into three sections:
  1) Set up the environment.
  2) Set up the functions for detecting hallucinations.
  3) Test out the functions.

## 1) Set up the envionment.

In [3]:
#!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 638.0 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.1/123.1 kB 1.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.8 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 5.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 37.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 10.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 17.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.8 MB

In [5]:
from sentence_transformers import CrossEncoder

model = CrossEncoder('vectara/hallucination_evaluation_model')

## 2) Set up the functions for detecting hallucinations.

In [6]:
def get_hallucination_score(source, generation):
    '''
    A score of less than 0.5 indicates a likely hallucination.
    Note that the context length of the model is 512 tokens across both documents.
    '''
    scores = model.predict([source,generation])
    return scores

## 3) Test out the functions.

In [17]:
#A flag to run the tests - defaults to off so that this functions can be called by other scripts.
RUN_EXAMPLES = False
if __name__ == '__main__':
    RUN_EXAMPLES = True

In [18]:
if RUN_EXAMPLES:
    original_text = "A man walks into a bar and buys a drink"
    generated_text = "A man swigs alcohol at a pub"

    is_hallucination = False
    score = get_hallucination_score(original_text,generated_text)
    if score<0.5:is_hallucination=True

    print ("Original:  ",original_text)
    print ("Generated: ",generated_text)
    if is_hallucination:
        print("The generated text likely contains a hallucination. (score of %s)"%(round(score,3)))
    else:
        print("The generated text likely does NOT contain a hallucination. (score of %s)"%(round(score,3)))

Original:   A man walks into a bar and buys a drink
Generated:  A man swigs alcohol at a pub
The generated text likely does NOT contain a hallucination. (score of 0.534)
